In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import classification_report, f1_score
import pickle
import os.path
import plotly.offline as pyo
import plotly.graph_objs as go
import spacy
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
nlp = spacy.load('en_core_web_sm')


# stop words built in spacy (english)
print(nlp.Defaults.stop_words)

{'along', '‘s', 'using', 'beforehand', 'very', 'anyone', 'hundred', '‘ve', '‘m', 'sixty', 'most', 'together', 'after', 'hers', 'top', 'without', 'does', 'never', 'into', 'among', 'neither', 'make', 'whether', 'wherever', 'alone', 'namely', 'indeed', 'she', 'whoever', "'ve", 'forty', '’ve', 'above', 'do', 'another', 'every', 'here', 'enough', 'nevertheless', 'itself', 'once', 'thereafter', 'whereupon', 'as', 'about', 'back', 'various', 'therefore', 'least', 'unless', 'latter', '’m', 'themselves', 'seemed', 'these', 'however', 'show', 'would', 'should', 'done', 'seem', 'had', 'been', 're', 'throughout', 'other', 'ca', 'get', 'why', 'it', 'then', '’s', 'each', 'somewhere', 'since', 'whither', 'under', 'hereupon', 'besides', 'himself', 'former', 'regarding', 'others', 'beside', 'ever', 'upon', 'due', 'them', 'us', 'though', 'those', 'is', 'onto', 'whereas', 'just', 'ourselves', 'same', 'will', 'nobody', 'made', 'although', 'further', 'this', 'side', 'keep', 'no', 'take', 'hence', 'well', '

In [4]:
s_stemmer = SnowballStemmer(language='english')

In [5]:
count_vect = CountVectorizer()

In [6]:
df = pd.read_csv('C:\\Users\\namit\\Downloads\\MBTI 500.csv')
df.head()

,posts,type
0,know intj tool use interaction people excuse a...,INTJ
1,rap music ehh opp yeah know valid well know fa...,INTJ
2,preferably p hd low except wew lad video p min...,INTJ
3,drink like wish could drink red wine give head...,INTJ
4,space program ah bad deal meing freelance max ...,INTJ


In [7]:
df['posts'][0]

'know intj tool use interaction people excuse antisocial truly enlighten mastermind know would count pet peeze something time matter people either whether group people mall never see best friend sit outside conversation jsut listen want interject sit formulate say wait inject argument thought find fascinate sit watch people talk people fascinate sit class watch different people find intrigue dad intj u stand look like line safeway watch people home talk people like think military job people voluntarily go job important show deference endanger live glorify way civilian think pretty ignorant general think military necessary defense mechanism political tactic feel like u specifically invest much money could put money education whatnot though personally sound budget aernative really comment one way base two politician eye year ago come name somewhat important kinda role model nowadays pick keep score individual level mean little vary accord number condition day may score high others low sw

In [8]:
df['type'].unique()

array(['INTJ', 'INTP', 'ISFJ', 'ISFP', 'ISTJ', 'ISTP', 'ENFJ', 'ENFP',
       'ENTJ', 'ENTP', 'ESFJ', 'ESFP', 'ESTJ', 'ESTP', 'INFJ', 'INFP'],
      dtype=object)

In [9]:
df.isnull().sum()

posts    0
type     0
dtype: int64

In [10]:
df_bar_chart=df.groupby('type').count()


trace1 = go.Bar(x=df_bar_chart.index, y=df_bar_chart['posts'])

data = [trace1]
layout = go.Layout(title='MBTI # Classified Posts per Type')

fig = go.Figure(data=data, layout=layout)

fig.show()

In [11]:
recreate_model=False

In [12]:
filename = 'mbti_svm_v2.sav'

In [13]:
if not os.path.isfile(filename):
    recreate_model=True

In [14]:
X = df['posts'] # features
y = df['type']  # labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
if recreate_model:    
    
    # Creating an instance to vectorizer:
    vectorizer = TfidfVectorizer()
    
    # Training the vectorizer:
    X_train_tfidf = vectorizer.fit_transform(X_train)
    
    # Training the classifier:
    clf = LinearSVC()
    clf.fit(X_train_tfidf, y_train)
    
    # Pipelining the vectorizer and the classifier
    text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])
    text_clf.fit(X_train, y_train)
    
    # saving the model to disk
    pickle.dump(text_clf, open(filename, 'wb'))

# If there is no need to recreate the model, just open the file from the disk    
else:
    # loading the model from disk
    text_clf = pickle.load(open(filename, 'rb'))

In [16]:
predictions = text_clf.predict(X_test)

In [17]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

        ENFJ       0.84      0.58      0.69       319
        ENFP       0.82      0.78      0.80      1249
        ENTJ       0.90      0.80      0.84       577
        ENTP       0.86      0.83      0.84      2324
        ESFJ       0.83      0.45      0.59        33
        ESFP       0.88      0.48      0.62        75
        ESTJ       0.90      0.84      0.87       105
        ESTP       0.95      0.90      0.92       398
        INFJ       0.81      0.84      0.83      2954
        INFP       0.80      0.82      0.81      2391
        INTJ       0.83      0.87      0.85      4531
        INTP       0.84      0.87      0.86      5033
        ISFJ       0.80      0.61      0.69       132
        ISFP       0.81      0.60      0.69       161
        ISTJ       0.86      0.68      0.76       253
        ISTP       0.89      0.79      0.84       679

    accuracy                           0.84     21214
   macro avg       0.85   

In [18]:
print(f"Overall accuracy of the model: {round(metrics.accuracy_score(y_test, predictions),2)}")

Overall accuracy of the model: 0.84


In [19]:
prediction = text_clf.predict(['I am a big fan of the MBTI'])
print(prediction)

['ESTP']
